In [ ]:
import pandas as pd

df = pd.read_csv("your_file.csv")  # Use actual file path

# Ensure numbers are properly parsed
numeric_cols = ['open', 'last', 'high', 'low', 'change', 'bPrice', 'bQty', 'sPrice', 'sQty',
                'ltq', 'avgPrice', 'quotes', 'ttq', 'totalBuyQt', 'totalSellQ', 'ttv',
                'VolumeC', 'OI', 'CHNGOI', 'lowerCktLm', 'upperCktLm', 'close']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# 1. 🔎 Large Deals: Top 2% by total traded value or volume
large_deals = df[(df['ttv'] > df['ttv'].quantile(0.98)) | (df['VolumeC'] > df['VolumeC'].quantile(0.98))]

# 2. 📈 Strong Buy Pressure: Bid > Ask, high last traded quantity
strong_buys = df[(df['bQty'] > df['sQty']) & (df['ltq'] > 10000)]

# 3. 🔄 OI Build-up: High OI + OI change
oi_build = df[(df['OI'] > df['OI'].quantile(0.9)) & (df['CHNGOI'] > 0)]

# 4. 🚀 Trending Stocks: Strong price momentum
trending_up = df[df['change'] > 5]  # >5% gain
trending_down = df[df['change'] < -5]  # >5% loss

# 5. 📊 Combined Signal: Intersection of large volume + buy pressure + OI build-up
smart_signals = large_deals.merge(strong_buys, on='symbol').merge(oi_build, on='symbol')

# Show relevant insights
print("Top Large Deals:\n", large_deals[['symbol', 'ttv', 'VolumeC', 'last']].sort_values(by='ttv', ascending=False).head())
print("\nStrong Buy Pressure:\n", strong_buys[['symbol', 'bQty', 'sQty', 'ltq']].head())
print("\nOI Build-Up:\n", oi_build[['symbol', 'OI', 'CHNGOI']].head())
print("\nTrending Stocks:\n", trending_up[['symbol', 'change']].head())


In [ ]:
import pandas as pd

# === Load File ===
import pandas as pd

# === Load File ===
file_path = "/content/extracted/18-07-2025_0.txt"  # Replace this if the filename is different

# Define column names based on the data structure
columns = [
    "symbol", "open", "last", "high", "low", "change", "bPrice", "bQty", "sPrice", "sQty",
    "ltq", "avgPrice", "quotes", "ttq", "totalBuyQt", "totalSellQ", "ttv", "trend",
    "lowerCktLm", "upperCktLm", "ltt", "close", "exchange", "stock_name", "VolumeC", "OI",
    "CHNGOI", "product_type", "expiry_date", "strike_price"
]

# Load the CSV without a header and assign the defined column names
df = pd.read_csv(file_path, header=None, names=columns)


# === Convert Required Columns ===
numeric_cols = ['open', 'last', 'high', 'low', 'change', 'bPrice', 'bQty', 'sPrice',
                'sQty', 'ltq', 'avgPrice', 'ttq', 'totalBuyQt', 'totalSellQ',
                'ttv', 'VolumeC', 'OI', 'CHNGOI', 'lowerCktLm', 'upperCktLm', 'close']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

df.drop_duplicates(inplace=True)

#df = pd.concat(df_list, ignore_index=True).drop_duplicates()
df = df.sort_values('ltt').groupby('symbol').tail(1).reset_index(drop=True)

# === Compute Derived Fields ===
df['Change%'] = ((df['last'] - df['open']) / df['open']) * 100

# === Sentiment Classification ===
def classify_sentiment(row):
    if row['change'] > 0 and row['CHNGOI'] > 0:
        return "Bullish Buildup"
    elif row['change'] < 0 and row['CHNGOI'] > 0:
        return "Bearish Buildup"
    elif row['change'] > 0 and row['CHNGOI'] < 0:
        return "Short Covering"
    elif row['change'] < 0 and row['CHNGOI'] < 0:
        return "Long Unwinding"
    else:
        return "Neutral"

df['Sentiment'] = df.apply(classify_sentiment, axis=1)

# === 🔥 Strongest / ❄️ Weakest ===
strongest = df.sort_values(by='Change%', ascending=False).head(10)
weakest = df.sort_values(by='Change%').head(10)

# === 📉 Sector-wise Summary ===
if 'stock_name' in df.columns:
    sector_summary = df.groupby('stock_name')['Sentiment'].value_counts().unstack(fill_value=0)
else:
    sector_summary = pd.DataFrame()

# === 📅 Market Outlook ===
bulls = (df['Sentiment'] == 'Bullish Buildup').sum()
bears = (df['Sentiment'] == 'Bearish Buildup').sum()
short_covering = (df['Sentiment'] == 'Short Covering').sum()
unwinding = (df['Sentiment'] == 'Long Unwinding').sum()

net_strength = (bulls + short_covering) - (bears + unwinding)
outlook = "📈 Bullish Bias" if net_strength > 0 else "📉 Bearish Bias" if net_strength < 0 else "➖ Neutral"

# === 🏦 Large Deals Detection ===
high_volume = df['VolumeC'].quantile(0.98)
high_ttv = df['ttv'].quantile(0.98)
large_deals = df[(df['VolumeC'] >= high_volume) | (df['ttv'] >= high_ttv)]
large_deals = large_deals.sort_values(by='ttv', ascending=False).head(10)

# === 🧲 Large Investor Buy/Sell Pressure ===
buy_dominant = df[(df['bQty'] > df['sQty']) & (df['ltq'] > 10000)].sort_values(by='bQty', ascending=False).head(10)
sell_dominant = df[(df['sQty'] > df['bQty']) & (df['ltq'] > 10000)].sort_values(by='sQty', ascending=False).head(10)

# === 📊 Print Results ===
print("\n🔥 Strongest Stocks:\n", strongest[['symbol', 'Change%', 'VolumeC', 'OI', 'Sentiment']])
print("\n❄️ Weakest Stocks:\n", weakest[['symbol', 'Change%', 'VolumeC', 'OI', 'Sentiment']])
print("\n📅 Predicted Market Outlook:", outlook)
print(f"Net Strength Score: {net_strength} (Bulls + SC vs Bears + UW)")

if not sector_summary.empty:
    print("\n📉 Sector-wise Sentiment Summary:\n", sector_summary)

print("\n💰 Top Large Deals (High Volume/TTv):\n", large_deals[['symbol', 'last', 'ttv', 'VolumeC', 'Change%']])
print("\n🏦 Potential Large Buyer Activity:\n", buy_dominant[['symbol', 'bQty', 'sQty', 'ltq', 'last']])
print("\n🔻 Potential Large Seller Activity:\n", sell_dominant[['symbol', 'bQty', 'sQty', 'ltq', 'last']])



🔥 Strongest Stocks:
          symbol  Change%  VolumeC  OI Sentiment
0    1.1!533202      inf      0.0 NaN   Neutral
5    1.1!533896      inf      0.0 NaN   Neutral
432  1.1!524031      inf      0.0 NaN   Neutral
438  1.1!511441      inf      0.0 NaN   Neutral
542  1.1!530765      inf      0.0 NaN   Neutral
609  1.1!540728      inf      0.0 NaN   Neutral
6    1.1!538119      inf      0.0 NaN   Neutral
11   1.1!532468      inf      0.0 NaN   Neutral
257  1.1!502937      inf      0.0 NaN   Neutral
12   1.1!532662      inf      0.0 NaN   Neutral

❄️ Weakest Stocks:
          symbol  Change%    VolumeC  OI Sentiment
588  1.1!530291   -100.0    5644.05 NaN   Neutral
593  1.1!530617   -100.0  318000.00 NaN   Neutral
587  1.1!531280   -100.0    5617.24 NaN   Neutral
540  1.1!506543   -100.0     124.85 NaN   Neutral
534  1.1!513642   -100.0   30095.10 NaN   Neutral
603  1.1!513544   -100.0    5969.92 NaN   Neutral
630  1.1!519174   -100.0   10471.63 NaN   Neutral
628  1.1!511451   -100.0    6

In [ ]:
import pandas as pd

# === Load File ===
file_path = "/content/extracted/18-07-2025_0.txt"  # Replace this if the filename is different

# Define column names based on the data structure
columns = [
    "symbol", "open", "last", "high", "low", "change", "bPrice", "bQty", "sPrice", "sQty",
    "ltq", "avgPrice", "quotes", "ttq", "totalBuyQt", "totalSellQ", "ttv", "trend",
    "lowerCktLm", "upperCktLm", "ltt", "close", "exchange", "stock_name", "VolumeC", "OI",
    "CHNGOI", "product_type", "expiry_date", "strike_price"
]

# Load the CSV without a header and assign the defined column names
df = pd.read_csv(file_path, header=None, names=columns)

# === Convert Required Columns ===
numeric_cols = ['open', 'last', 'high', 'low', 'change', 'bPrice', 'bQty', 'sPrice',
                'sQty', 'ltq', 'avgPrice', 'ttq', 'totalBuyQt', 'totalSellQ',
                'ttv', 'VolumeC', 'OI', 'CHNGOI', 'lowerCktLm', 'upperCktLm', 'close']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

df.drop_duplicates(inplace=True)

#df = pd.concat(df_list, ignore_index=True).drop_duplicates()
df = df.sort_values('ltt').groupby('symbol').tail(1).reset_index(drop=True)

# === Compute Derived Fields ===
df['Change%'] = ((df['last'] - df['open']) / df['open']) * 100

# === Sentiment Classification ===
def classify_sentiment(row):
    if row['change'] > 0 and row['CHNGOI'] > 0:
        return "Bullish Buildup"
    elif row['change'] < 0 and row['CHNGOI'] > 0:
        return "Bearish Buildup"
    elif row['change'] > 0 and row['CHNGOI'] < 0:
        return "Short Covering"
    elif row['change'] < 0 and row['CHNGOI'] < 0:
        return "Long Unwinding"
    else:
        return "Neutral"

df['Sentiment'] = df.apply(classify_sentiment, axis=1)

# === 🔥 Strongest / ❄️ Weakest ===
strongest = df.sort_values(by='Change%', ascending=False).head(10)
weakest = df.sort_values(by='Change%').head(10)

# === 📉 Sector-wise Summary ===
if 'stock_name' in df.columns:
    sector_summary = df.groupby('stock_name')['Sentiment'].value_counts().unstack(fill_value=0)
else:
    sector_summary = pd.DataFrame()

# === 📅 Market Outlook ===
bulls = (df['Sentiment'] == 'Bullish Buildup').sum()
bears = (df['Sentiment'] == 'Bearish Buildup').sum()
short_covering = (df['Sentiment'] == 'Short Covering').sum()
unwinding = (df['Sentiment'] == 'Long Unwinding').sum()

net_strength = (bulls + short_covering) - (bears + unwinding)
outlook = "📈 Bullish Bias" if net_strength > 0 else "📉 Bearish Bias" if net_strength < 0 else "➖ Neutral"

# === 💰 Large Deals Detection ===
high_volume = df['VolumeC'].quantile(0.98)
high_ttv = df['ttv'].quantile(0.98)
large_deals = df[(df['VolumeC'] >= high_volume) | (df['ttv'] >= high_ttv)]
large_deals = large_deals.sort_values(by='ttv', ascending=False).head(10)

# === 🏦 Large Investor Buy/Sell Pressure ===
buy_dominant = df[(df['bQty'] > df['sQty']) & (df['ltq'] > 10000)].sort_values(by='bQty', ascending=False).head(10)
sell_dominant = df[(df['sQty'] > df['bQty']) & (df['ltq'] > 10000)].sort_values(by='sQty', ascending=False).head(10)

# === 🧭 Market Pressure ===
df['BuyPressure'] = (df['bQty'] > df['sQty']) & (df['ltq'] > 10000)
df['SellPressure'] = (df['sQty'] > df['bQty']) & (df['ltq'] > 10000)
buy_count = df['BuyPressure'].sum()
sell_count = df['SellPressure'].sum()

if buy_count > sell_count:
    pressure = "🟢 Market Under Buy Pressure"
elif sell_count > buy_count:
    pressure = "🔴 Market Under Sell Pressure"
else:
    pressure = "⚖️ Balanced Pressure"

# === 📊 Print Results ===
print("\n🔥 Strongest Stocks:\n", strongest[['symbol', 'Change%', 'VolumeC', 'OI', 'Sentiment']])
print("\n❄️ Weakest Stocks:\n", weakest[['symbol', 'Change%', 'VolumeC', 'OI', 'Sentiment']])
print("\n📅 Predicted Market Outlook:", outlook)
print(f"Net Strength Score: {net_strength} (Bulls + SC vs Bears + UW)")

if not sector_summary.empty:
    print("\n📉 Sector-wise Sentiment Summary:\n", sector_summary)

print("\n💰 Top Large Deals (High Volume/TTv):\n", large_deals[['symbol', 'last', 'ttv', 'VolumeC', 'Change%']])
print("\n🏦 Potential Large Buyer Activity:\n", buy_dominant[['symbol', 'bQty', 'sQty', 'ltq', 'last']])
print("\n🔻 Potential Large Seller Activity:\n", sell_dominant[['symbol', 'bQty', 'sQty', 'ltq', 'last']])
print("\n🧭 Market Pressure Analysis:")
print(f"Buy Dominant Stocks: {buy_count}")
print(f"Sell Dominant Stocks: {sell_count}")
print(f"Overall Market Pressure: {pressure}")


# === 📊 Composite Market Strength Score Strategy ===
advancers = df[df['change'] > 0].shape[0]
decliners = df[df['change'] < 0].shape[0]
adr = advancers / (decliners + 1)

# VWAP Bias
df['VWAP'] = df['ttv'] / df['VolumeC']
df['VWAP_Bias'] = df['last'] > df['VWAP']
vwap_bias_count = df['VWAP_Bias'].sum()

# Composite Score
score = 0
if net_strength > 0: score += 1
if adr > 1: score += 1
if vwap_bias_count > len(df) * 0.6: score += 1
if bulls > bears: score += 1
if short_covering > unwinding: score += 1

print("\n🧠 Composite Strategy Signals:")
print(f"Advance/Decline Ratio (ADR): {adr:.2f}")
print(f"Stocks Above VWAP: {vwap_bias_count}/{len(df)}")
print(f"Net Sentiment Strength Score: {net_strength}")
print(f"Composite Signal Score: {score}/5")

# Final Prediction
if score >= 4:
    print("📈 High Probability Bullish Day Tomorrow")
elif score <= 1:
    print("📉 High Probability Bearish Day Tomorrow")
else:
    print("⚖️ Rangebound or Mixed Sentiment Expected Tomorrow")


🔥 Strongest Stocks:
          symbol  Change%  VolumeC  OI Sentiment
0    1.1!533202      inf      0.0 NaN   Neutral
5    1.1!533896      inf      0.0 NaN   Neutral
432  1.1!524031      inf      0.0 NaN   Neutral
438  1.1!511441      inf      0.0 NaN   Neutral
542  1.1!530765      inf      0.0 NaN   Neutral
609  1.1!540728      inf      0.0 NaN   Neutral
6    1.1!538119      inf      0.0 NaN   Neutral
11   1.1!532468      inf      0.0 NaN   Neutral
257  1.1!502937      inf      0.0 NaN   Neutral
12   1.1!532662      inf      0.0 NaN   Neutral

❄️ Weakest Stocks:
          symbol  Change%    VolumeC  OI Sentiment
588  1.1!530291   -100.0    5644.05 NaN   Neutral
593  1.1!530617   -100.0  318000.00 NaN   Neutral
587  1.1!531280   -100.0    5617.24 NaN   Neutral
540  1.1!506543   -100.0     124.85 NaN   Neutral
534  1.1!513642   -100.0   30095.10 NaN   Neutral
603  1.1!513544   -100.0    5969.92 NaN   Neutral
630  1.1!519174   -100.0   10471.63 NaN   Neutral
628  1.1!511451   -100.0    6

In [ ]:
import pandas as pd
import numpy as np

# === Load Market Data ===
file_path = "18-07-2025_0.txt"  # Change this to your input file
df = pd.read_csv(file_path)

# Convert numeric columns
numeric_cols = ['open', 'last', 'high', 'low', 'change', 'bPrice', 'bQty', 'sPrice',
                'sQty', 'ltq', 'avgPrice', 'ttq', 'totalBuyQt', 'totalSellQ',
                'ttv', 'VolumeC', 'OI', 'CHNGOI', 'lowerCktLm', 'upperCktLm', 'close']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# === VWAP Bias ===
df['VWAP'] = df['ttv'] / df['VolumeC']
vwap_bias = (df['last'] > df['VWAP']).sum()
vwap_score = 1 if vwap_bias > len(df) * 0.6 else 0

# === Order Book Imbalance ===
order_imbalance_score = 1 if (df['bQty'] > df['sQty']).sum() > len(df) * 0.6 else 0

# === Smart Money Flow Index (SMFI Proxy) ===
df['close_strength'] = df['last'] - df['open']
smfi_score = 1 if df['close_strength'].mean() > 0 else 0

# === Volume Profile Break Proxy ===
df['Range'] = df['high'] - df['low']
range_breaks = df[df['last'] >= df['high']]
volume_break_score = 1 if len(range_breaks) > len(df) * 0.2 else 0

# === Global Cue Input (mocked here, replace with real data if available) ===
sgx_nifty_up = True
usd_inr_weak = True
global_score = 1 if sgx_nifty_up and usd_inr_weak else 0

# === Composite Institutional Score ===
total_score = vwap_score + order_imbalance_score + smfi_score + volume_break_score + global_score

print("\n🧠 SMART INSTITUTIONAL MODEL SUMMARY:")
print(f"VWAP Bias Score: {vwap_score}")
print(f"Order Book Imbalance Score: {order_imbalance_score}")
print(f"Smart Money Flow Proxy Score: {smfi_score}")
print(f"Volume Breakout Score: {volume_break_score}")
print(f"Global Cue Score: {global_score}")
print(f"Total Score: {total_score}/5")

# === Final Prediction ===
if total_score >= 4:
    print("📈 Institutional Signal: STRONG BULLISH BIAS for Tomorrow")
elif total_score <= 1:
    print("📉 Institutional Signal: STRONG BEARISH BIAS for Tomorrow")
else:
    print("⚖️ Institutional Signal: NEUTRAL or RANGEBOUND Market")


In [ ]:
import pandas as pd
import numpy as np

# === Load Market Data ===
file_path = "18-07-2025_0.txt"  # Change this to your file path
df = pd.read_csv(file_path)

# === Convert numeric columns ===
numeric_cols = [
    'open', 'last', 'high', 'low', 'change', 'bPrice', 'bQty', 'sPrice',
    'sQty', 'ltq', 'avgPrice', 'ttq', 'totalBuyQt', 'totalSellQ',
    'ttv', 'VolumeC', 'OI', 'CHNGOI', 'lowerCktLm', 'upperCktLm', 'close'
]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# === VWAP Bias ===
df['VWAP'] = df['ttv'] / df['VolumeC']
vwap_bias = (df['last'] > df['VWAP']).sum()
vwap_score = 1 if vwap_bias > len(df) * 0.6 else 0

# === Order Book Imbalance (Smart Money Accumulation) ===
order_imbalance_score = 1 if (df['bQty'] > df['sQty']).sum() > len(df) * 0.6 else 0

# === Smart Money Flow Index (SMFI Proxy) ===
df['close_strength'] = df['last'] - df['open']
smfi_score = 1 if df['close_strength'].mean() > 0 else 0

# === Volume Profile Breakout Proxy ===
df['Range'] = df['high'] - df['low']
range_breaks = df[df['last'] >= df['high']]
volume_break_score = 1 if len(range_breaks) > len(df) * 0.2 else 0

# === Global Cue Inputs (mock values for now) ===
sgx_nifty_up = True         # ✅ Replace with API input or manual toggle
usd_inr_weak = True         # ✅ Replace with actual data if needed
global_score = 1 if sgx_nifty_up and usd_inr_weak else 0

# === Composite Institutional Strength Score ===
total_score = (
    vwap_score +
    order_imbalance_score +
    smfi_score +
    volume_break_score +
    global_score
)

# === 📊 Print Results ===
print("\n🧠 SMART INSTITUTIONAL MODEL SUMMARY:")
print(f"VWAP Bias Score: {vwap_score}")
print(f"Order Book Imbalance Score: {order_imbalance_score}")
print(f"Smart Money Flow Proxy Score: {smfi_score}")
print(f"Volume Breakout Score: {volume_break_score}")
print(f"Global Cue Score: {global_score}")
print(f"➡️ Total Score: {total_score}/5")

# === Final Prediction ===
if total_score >= 4:
    print("📈 Institutional Signal: STRONG BULLISH BIAS for Tomorrow")
elif total_score <= 1:
    print("📉 Institutional Signal: STRONG BEARISH BIAS for Tomorrow")
else:
    print("⚖️ Institutional Signal: NEUTRAL or RANGEBOUND Market")


In [ ]:
import pandas as pd
import numpy as np
import glob
import os

import zipfile
import os

# === Path to your ZIP file ===
zip_path = "18-07-2025_0.zip"  # Ensure it's in the same folder or provide full path
extract_to = "extracted"

# === Create folder if needed ===
os.makedirs(extract_to, exist_ok=True)

# === Extract all files ===
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# === List extracted files ===
print(f"✅ Extracted to: {extract_to}")
print("📄 Files:")
for file in os.listdir(extract_to):
    print(" -", file)


# === 🔄 Load and Combine All Market Files ===
folder = "extracted"  # Or specify full path
files = glob.glob(os.path.join(folder, "18-07-2025_*.txt"))

columns = [
    "symbol", "open", "last", "high", "low", "change", "bPrice", "bQty", "sPrice", "sQty",
    "ltq", "avgPrice", "quotes", "ttq", "totalBuyQt", "totalSellQ", "ttv", "trend",
    "lowerCktLm", "upperCktLm", "ltt", "close", "exchange", "stock_name", "VolumeC", "OI",
    "CHNGOI", "product_type", "expiry_date", "strike_price"
]


df_list = []
for file in files:
    try:
        # Specify no header and provide column names
        df_temp = pd.read_csv(file, header=None, names=columns)
        df_list.append(df_temp)
    except Exception as e:
        print(f"Error reading {file}: {e}")

df = pd.concat(df_list, ignore_index=True)
print(f"\n📁 Loaded {len(files)} files, Total rows: {len(df)}")

# === Convert numeric columns ===
numeric_cols = ['open', 'last', 'high', 'low', 'change', 'bPrice', 'bQty', 'sPrice',
                'sQty', 'ltq', 'avgPrice', 'quotes', 'ttq', 'totalBuyQt', 'totalSellQ',
                'ttv', 'VolumeC', 'OI', 'CHNGOI', 'lowerCktLm', 'upperCktLm', 'close']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# df = pd.concat(df_list, ignore_index=True).drop_duplicates()
df = df.sort_values('ltt').groupby('symbol').tail(1).reset_index(drop=True)
# === VWAP Bias ===
df['VWAP'] = df['ttv'] / df['VolumeC']
vwap_bias = (df['last'] > df['VWAP']).sum()
vwap_score = 1 if vwap_bias > len(df) * 0.6 else 0

# === Order Book Imbalance (Smart Money Accumulation) ===
order_imbalance_score = 1 if (df['bQty'] > df['sQty']).sum() > len(df) * 0.6 else 0

# === Smart Money Flow Index (SMFI Proxy) ===
df['close_strength'] = df['last'] - df['open']
smfi_score = 1 if df['close_strength'].mean() > 0 else 0

# === Volume Profile Breakout Proxy ===
df['Range'] = df['high'] - df['low']
range_breaks = df[df['last'] >= df['high']]
volume_break_score = 1 if len(range_breaks) > len(df) * 0.2 else 0

# === Global Cue Inputs (mock values) ===
sgx_nifty_up = True         # ✅ Replace with API or real values
usd_inr_weak = True
global_score = 1 if sgx_nifty_up and usd_inr_weak else 0

# === Composite Institutional Strength Score ===
total_score = (
    vwap_score +
    order_imbalance_score +
    smfi_score +
    volume_break_score +
    global_score
)

# === 📊 Print Results ===
print("\n🧠 SMART INSTITUTIONAL MODEL SUMMARY:")
print(f"VWAP Bias Score: {vwap_score}")
print(f"Order Book Imbalance Score: {order_imbalance_score}")
print(f"Smart Money Flow Proxy Score: {smfi_score}")
print(f"Volume Breakout Score: {volume_break_score}")
print(f"Global Cue Score: {global_score}")
print(f"➡️ Total Score: {total_score}/5")

# === Final Prediction ===
if total_score >= 4:
    print("📈 Institutional Signal: STRONG BULLISH BIAS for Tomorrow")
elif total_score <= 1:
    print("📉 Institutional Signal: STRONG BEARISH BIAS for Tomorrow")
else:
    print("⚖️ Institutional Signal: NEUTRAL or RANGEBOUND Market")

✅ Extracted to: extracted
📄 Files:
 - 18-07-2025_0.txt

📁 Loaded 1 files, Total rows: 1574071

🧠 SMART INSTITUTIONAL MODEL SUMMARY:
VWAP Bias Score: 0
Order Book Imbalance Score: 0
Smart Money Flow Proxy Score: 0
Volume Breakout Score: 1
Global Cue Score: 1
➡️ Total Score: 2/5
⚖️ Institutional Signal: NEUTRAL or RANGEBOUND Market


In [ ]:
import pandas as pd
import numpy as np
import glob
import os

import zipfile
import os

# === Path to your ZIP file ===
zip_path = "18-07-2025_0.zip"  # Ensure it's in the same folder or provide full path
extract_to = "extracted"

# === Create folder if needed ===
os.makedirs(extract_to, exist_ok=True)

# === Extract all files ===
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# === List extracted files ===
print(f"✅ Extracted to: {extract_to}")
print("📄 Files:")
for file in os.listdir(extract_to):
    print(" -", file)


# === 🔄 Load and Combine All Market Files ===
folder = "extracted"  # Or specify full path
files = glob.glob(os.path.join(folder, "18-07-2025_*.txt"))

columns = [
    "symbol", "open", "last", "high", "low", "change", "bPrice", "bQty", "sPrice", "sQty",
    "ltq", "avgPrice", "quotes", "ttq", "totalBuyQt", "totalSellQ", "ttv", "trend",
    "lowerCktLm", "upperCktLm", "ltt", "close", "exchange", "stock_name", "VolumeC", "OI",
    "CHNGOI", "product_type", "expiry_date", "strike_price"
]


df_list = []
for file in files:
    try:
        # Specify no header and provide column names
        df_temp = pd.read_csv(file, header=None, names=columns)
        df_list.append(df_temp)
    except Exception as e:
        print(f"Error reading {file}: {e}")

df = pd.concat(df_list, ignore_index=True)
print(f"\n📁 Loaded {len(files)} files, Total rows: {len(df)}")

# === Convert numeric columns ===
numeric_cols = ['open', 'last', 'high', 'low', 'change', 'bPrice', 'bQty', 'sPrice',
                'sQty', 'ltq', 'avgPrice', 'quotes', 'ttq', 'totalBuyQt', 'totalSellQ',
                'ttv', 'VolumeC', 'OI', 'CHNGOI', 'lowerCktLm', 'upperCktLm', 'close']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')


df = pd.concat(df_list, ignore_index=True).drop_duplicates()
numeric_cols = ['open', 'last', 'high', 'low', 'bPrice', 'sPrice', 'bQty', 'sQty', 'ltq', 'VolumeC']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# === Tick-based Buy/Sell Pressure ===
df['delta_price'] = df['last'].diff()
df['buy_volume'] = np.where(df['delta_price'] > 0, df['ltq'], 0)
df['sell_volume'] = np.where(df['delta_price'] < 0, df['ltq'], 0)

total_buy = df['buy_volume'].sum()
total_sell = df['sell_volume'].sum()

# === Order Imbalance ===
df['order_imbalance'] = (df['bQty'] - df['sQty']) / (df['bQty'] + df['sQty'])
net_order_imbalance = df['order_imbalance'].mean()

# === Money Flow Index Proxy ===
df['typical_price'] = (df['high'] + df['low'] + df['last']) / 3
df['money_flow'] = df['typical_price'] * df['VolumeC']
df['raw_flow'] = np.where(df['typical_price'] > df['typical_price'].shift(), df['money_flow'], -df['money_flow'])
mfi_score = df['raw_flow'].rolling(window=14).sum().iloc[-1]

# === Accumulation/Distribution ===
df['clv'] = ((df['last'] - df['low']) - (df['high'] - df['last'])) / (df['high'] - df['low'])
df['clv'] = df['clv'].fillna(0)
df['ad_volume'] = df['clv'] * df['VolumeC']
ad_trend = df['ad_volume'].cumsum().iloc[-1]

# === Sentiment Decision ===
print("\n💰 MONEY FLOW SENTIMENT ANALYSIS:")
print(f"Total Buy Volume: {total_buy}")
print(f"Total Sell Volume: {total_sell}")
print(f"Order Imbalance: {net_order_imbalance:.2f}")
print(f"MFI Score (14-tick): {mfi_score:.2f}")
print(f"Accumulation Trend: {ad_trend:.2f}")

if total_buy > total_sell and net_order_imbalance > 0.3 and mfi_score > 0:
    print("📈 Money Flow Sentiment: STRONG BUYING")
elif total_sell > total_buy and net_order_imbalance < -0.3 and mfi_score < 0:
    print("📉 Money Flow Sentiment: STRONG SELLING")
else:
    print("⚖️ Money Flow Sentiment: NEUTRAL / MIXED")


✅ Extracted to: extracted
📄 Files:
 - 18-07-2025_0.txt

📁 Loaded 1 files, Total rows: 1574071

💰 MONEY FLOW SENTIMENT ANALYSIS:
Total Buy Volume: 121055298
Total Sell Volume: 1467798216
Order Imbalance: -0.03
MFI Score (14-tick): 271800312863.35
Accumulation Trend: nan
⚖️ Money Flow Sentiment: NEUTRAL / MIXED


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: RuntimeWarning: invalid value encountered in accumulate
  return bound(*args, **kwds)


In [ ]:
import pandas as pd
import numpy as np
import glob
import os

import zipfile
import os

# === Path to your ZIP file ===
zip_path = "18-07-2025_0.zip"  # Ensure it's in the same folder or provide full path
extract_to = "extracted"

# === Create folder if needed ===
os.makedirs(extract_to, exist_ok=True)

# === Extract all files ===
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# === List extracted files ===
print(f"✅ Extracted to: {extract_to}")
print("📄 Files:")
for file in os.listdir(extract_to):
    print(" -", file)


# === 🔄 Load and Combine All Market Files ===
folder = "extracted"  # Or specify full path
files = glob.glob(os.path.join(folder, "18-07-2025_*.txt"))

columns = [
    "symbol", "open", "last", "high", "low", "change", "bPrice", "bQty", "sPrice", "sQty",
    "ltq", "avgPrice", "quotes", "ttq", "totalBuyQt", "totalSellQ", "ttv", "trend",
    "lowerCktLm", "upperCktLm", "ltt", "close", "exchange", "stock_name", "VolumeC", "OI",
    "CHNGOI", "product_type", "expiry_date", "strike_price"
]


df_list = []
for file in files:
    try:
        # Specify no header and provide column names
        df_temp = pd.read_csv(file, header=None, names=columns)
        df_list.append(df_temp)
    except Exception as e:
        print(f"Error reading {file}: {e}")

df = pd.concat(df_list, ignore_index=True)
print(f"\n📁 Loaded {len(files)} files, Total rows: {len(df)}")

# === Convert numeric columns ===
numeric_cols = ['open', 'last', 'high', 'low', 'change', 'bPrice', 'bQty', 'sPrice',
                'sQty', 'ltq', 'avgPrice', 'quotes', 'ttq', 'totalBuyQt', 'totalSellQ',
                'ttv', 'VolumeC', 'OI', 'CHNGOI', 'lowerCktLm', 'upperCktLm', 'close']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# === Tick-based Buy/Sell Pressure ===
df['delta_price'] = df['last'].diff()
df['buy_volume'] = np.where(df['delta_price'] > 0, df['ltq'], 0)
df['sell_volume'] = np.where(df['delta_price'] < 0, df['ltq'], 0)

total_buy = df['buy_volume'].sum()
total_sell = df['sell_volume'].sum()
sentiment_ratio = total_buy / (total_sell + 1)

# === Order Imbalance ===
df['order_imbalance'] = (df['bQty'] - df['sQty']) / (df['bQty'] + df['sQty'])
net_order_imbalance = df['order_imbalance'].mean()

# === Money Flow Index Proxy ===
df['typical_price'] = (df['high'] + df['low'] + df['last']) / 3
df['money_flow'] = df['typical_price'] * df['VolumeC']
df['raw_flow'] = np.where(df['typical_price'] > df['typical_price'].shift(), df['money_flow'], -df['money_flow'])
mfi_score = df['raw_flow'].rolling(window=14).sum().iloc[-1]
mfi_score_norm = np.log1p(abs(mfi_score)) * np.sign(mfi_score)

# === Accumulation/Distribution ===
df['range'] = (df['high'] - df['low']).replace(0, 0.0001)
df['clv'] = ((df['last'] - df['low']) - (df['high'] - df['last'])) / df['range']
df['clv'] = df['clv'].fillna(0)
df['ad_volume'] = df['clv'] * df['VolumeC']
ad_trend = df['ad_volume'].cumsum().iloc[-1]

# === Sentiment Decision ===
print("\n💰 MONEY FLOW SENTIMENT ANALYSIS (Fixed):")
print(f"Total Buy Volume: {total_buy}")
print(f"Total Sell Volume: {total_sell}")
print(f"Buy/Sell Sentiment Ratio: {sentiment_ratio:.3f}")
print(f"Order Imbalance: {net_order_imbalance:.2f}")
print(f"MFI Score (log-normalized): {mfi_score_norm:.2f}")
print(f"Accumulation Trend: {ad_trend:.2f}")

if sentiment_ratio < 0.2 and net_order_imbalance < -0.2 and mfi_score < 0:
    print("📉 Money Flow Sentiment: STRONG SELLING")
elif sentiment_ratio > 0.7 and net_order_imbalance > 0.2 and mfi_score > 0:
    print("📈 Money Flow Sentiment: STRONG BUYING")
else:
    print("⚖️ Money Flow Sentiment: NEUTRAL / MIXED")


✅ Extracted to: extracted
📄 Files:
 - 18-07-2025_0.txt

📁 Loaded 1 files, Total rows: 1574071

💰 MONEY FLOW SENTIMENT ANALYSIS (Fixed):
Total Buy Volume: 121062525
Total Sell Volume: 1468189820
Buy/Sell Sentiment Ratio: 0.082
Order Imbalance: -0.03
MFI Score (log-normalized): 26.33
Accumulation Trend: -1071038323623824.38
⚖️ Money Flow Sentiment: NEUTRAL / MIXED


In [ ]:

import pandas as pd
import numpy as np
import glob
import os

import zipfile
import os

# === Path to your ZIP file ===
zip_path = "18-07-2025_0.zip"  # Ensure it's in the same folder or provide full path
extract_to = "extracted"

# === Create folder if needed ===
os.makedirs(extract_to, exist_ok=True)

# === Extract all files ===
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# === List extracted files ===
print(f"✅ Extracted to: {extract_to}")
print("📄 Files:")
for file in os.listdir(extract_to):
    print(" -", file)


# === 🔄 Load and Combine All Market Files ===
folder = "extracted"  # Or specify full path
files = glob.glob(os.path.join(folder, "18-07-2025_*.txt"))


columns = [
    "symbol", "open", "last", "high", "low", "change", "bPrice", "bQty", "sPrice", "sQty",
    "ltq", "avgPrice", "quotes", "ttq", "totalBuyQt", "totalSellQ", "ttv", "trend",
    "lowerCktLm", "upperCktLm", "ltt", "close", "exchange", "stock_name", "VolumeC", "OI",
    "CHNGOI", "product_type", "expiry_date", "strike_price"
]

df_list = []
for file in files:
    df_temp = pd.read_csv(file, header=None, names=columns)
    df_list.append(df_temp)

df = pd.concat(df_list, ignore_index=True).drop_duplicates()
df['ltt'] = pd.to_datetime(df['ltt'], errors='coerce')

# === Define Smart Money Time Windows ===
opening_start = pd.to_datetime("09:15:00").time()
opening_end = pd.to_datetime("10:00:00").time()
closing_start = pd.to_datetime("14:45:00").time()
closing_end = pd.to_datetime("15:15:00").time()

df['time'] = df['ltt'].dt.time

# === Filter for Opening & Closing Smart Money Zones ===
df_opening = df[(df['time'] >= opening_start) & (df['time'] <= opening_end)]
df_closing = df[(df['time'] >= closing_start) & (df['time'] <= closing_end)]

# === Analyze Volume Clusters ===
opening_volumes = df_opening.groupby('symbol')['ltq'].sum().sort_values(ascending=False)
closing_volumes = df_closing.groupby('symbol')['ltq'].sum().sort_values(ascending=False)

# === Print Top Smart Money Symbols ===
print("\n⏳ SMART MONEY ZONE ANALYSIS")
print("🔁 Top 10 Opening Drive Volume Symbols (9:15–10:00):")
print(opening_volumes.head(10))

print("\n🔁 Top 10 Closing Accumulation Symbols (14:45–15:15):")
print(closing_volumes.head(10))


✅ Extracted to: extracted
📄 Files:
 - 18-07-2025_0.txt

⏳ SMART MONEY ZONE ANALYSIS
🔁 Top 10 Opening Drive Volume Symbols (9:15–10:00):
symbol
1.1!540614    308581567
1.1!531395     29447138
1.1!511700     24605206
1.1!511557     19984580
1.1!506642     19560054
1.1!505336     15645878
1.1!511012      3075990
1.1!512591      2275248
1.1!517554      2080586
1.1!524632      1723244
Name: ltq, dtype: int64

🔁 Top 10 Closing Accumulation Symbols (14:45–15:15):
symbol
1.1!511700    4074456
1.1!531395    3762176
1.1!506642    3624649
1.1!500389    2878862
1.1!505336    1758443
1.1!511557     968810
1.1!540614     743519
1.1!511012     513404
1.1!524174     355666
1.1!524632     238171
Name: ltq, dtype: int64


In [ ]:

import pandas as pd
import numpy as np
import glob
import os

import zipfile
import os

# === Path to your ZIP file ===
zip_path = "18-07-2025_0.zip"  # Ensure it's in the same folder or provide full path
extract_to = "extracted"

# === Create folder if needed ===
os.makedirs(extract_to, exist_ok=True)

# === Extract all files ===
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# === List extracted files ===
print(f"✅ Extracted to: {extract_to}")
print("📄 Files:")
for file in os.listdir(extract_to):
    print(" -", file)


# === 🔄 Load and Combine All Market Files ===
folder = "extracted"  # Or specify full path
files = glob.glob(os.path.join(folder, "18-07-2025_*.txt"))


columns = [
    "symbol", "open", "last", "high", "low", "change", "bPrice", "bQty", "sPrice", "sQty",
    "ltq", "avgPrice", "quotes", "ttq", "totalBuyQt", "totalSellQ", "ttv", "trend",
    "lowerCktLm", "upperCktLm", "ltt", "close", "exchange", "stock_name", "VolumeC", "OI",
    "CHNGOI", "product_type", "expiry_date", "strike_price"
]

df_list = []
for file in files:
    df_temp = pd.read_csv(file, header=None, names=columns)
    df_list.append(df_temp)

df = pd.concat(df_list, ignore_index=True).drop_duplicates()
df['ltt'] = pd.to_datetime(df['ltt'], errors='coerce')
df['time'] = df['ltt'].dt.time

# === Time Windows ===
opening_start = pd.to_datetime("09:15:00").time()
opening_end = pd.to_datetime("10:00:00").time()
closing_start = pd.to_datetime("14:45:00").time()
closing_end = pd.to_datetime("15:15:00").time()

df_opening = df[(df['time'] >= opening_start) & (df['time'] <= opening_end)]
df_closing = df[(df['time'] >= closing_start) & (df['time'] <= closing_end)]

def detect_sentiment(group):
    if len(group) < 2: return "Insufficient"
    price_change = group['last'].iloc[-1] - group['last'].iloc[0]
    avg_bqty = group['bQty'].mean()
    avg_sqty = group['sQty'].mean()
    if price_change > 0 and avg_bqty > avg_sqty:
        return "📈 Bullish"
    elif price_change < 0 and avg_sqty > avg_bqty:
        return "📉 Bearish"
    else:
        return "⚖️ Neutral"

# === Compute Volume & Sentiment ===
opening_summary = (
    df_opening.groupby('symbol')
    .agg(total_volume=('ltq', 'sum'))
    .sort_values('total_volume', ascending=False)
    .head(10)
)
opening_summary['Sentiment'] = opening_summary.index.to_series().apply(
    lambda sym: detect_sentiment(df_opening[df_opening['symbol'] == sym])
)

closing_summary = (
    df_closing.groupby('symbol')
    .agg(total_volume=('ltq', 'sum'))
    .sort_values('total_volume', ascending=False)
    .head(10)
)
closing_summary['Sentiment'] = closing_summary.index.to_series().apply(
    lambda sym: detect_sentiment(df_closing[df_closing['symbol'] == sym])
)

# === Display ===
print("\n⏳ SMART MONEY ZONE ANALYSIS WITH SENTIMENT")
print("\n🔁 Opening Drive (9:15–10:00):")
print(opening_summary)

print("\n🔁 Closing Session (14:45–15:15):")
print(closing_summary)

✅ Extracted to: extracted
📄 Files:
 - 18-07-2025_0.txt

⏳ SMART MONEY ZONE ANALYSIS WITH SENTIMENT

🔁 Opening Drive (9:15–10:00):
            total_volume   Sentiment
symbol                              
1.1!540614     308581567  ⚖️ Neutral
1.1!531395      29447138  ⚖️ Neutral
1.1!511700      24605206  ⚖️ Neutral
1.1!511557      19984580  ⚖️ Neutral
1.1!506642      19560054  ⚖️ Neutral
1.1!505336      15645878  ⚖️ Neutral
1.1!511012       3075990   📉 Bearish
1.1!512591       2275248  ⚖️ Neutral
1.1!517554       2080586  ⚖️ Neutral
1.1!524632       1723244  ⚖️ Neutral

🔁 Closing Session (14:45–15:15):
            total_volume   Sentiment
symbol                              
1.1!511700       4074456  ⚖️ Neutral
1.1!531395       3762176  ⚖️ Neutral
1.1!506642       3624649  ⚖️ Neutral
1.1!500389       2878862  ⚖️ Neutral
1.1!505336       1758443  ⚖️ Neutral
1.1!511557        968810  ⚖️ Neutral
1.1!540614        743519  ⚖️ Neutral
1.1!511012        513404  ⚖️ Neutral
1.1!524174        3556

In [ ]:

import pandas as pd
import numpy as np
import glob
import os

import zipfile
import os

# === Path to your ZIP file ===
zip_path = "18-07-2025_0.zip"  # Ensure it's in the same folder or provide full path
extract_to = "extracted"

# === Create folder if needed ===
os.makedirs(extract_to, exist_ok=True)

# === Extract all files ===
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# === List extracted files ===
print(f"✅ Extracted to: {extract_to}")
print("📄 Files:")
for file in os.listdir(extract_to):
    print(" -", file)


# === 🔄 Load and Combine All Market Files ===
folder = "extracted"  # Or specify full path
files = glob.glob(os.path.join(folder, "18-07-2025_*.txt"))


columns = [
    "symbol", "open", "last", "high", "low", "change", "bPrice", "bQty", "sPrice", "sQty",
    "ltq", "avgPrice", "quotes", "ttq", "totalBuyQt", "totalSellQ", "ttv", "trend",
    "lowerCktLm", "upperCktLm", "ltt", "close", "exchange", "stock_name", "VolumeC", "OI",
    "CHNGOI", "product_type", "expiry_date", "strike_price"
]

df_list = []
for file in files:
    df_temp = pd.read_csv(file, header=None, names=columns)
    df_list.append(df_temp)

df = pd.concat(df_list, ignore_index=True).drop_duplicates()
df['ltt'] = pd.to_datetime(df['ltt'], errors='coerce')
df['time'] = df['ltt'].dt.time

# === Filter Pre-Open Window: 09:00–09:15 ===
preopen_start = pd.to_datetime("09:00:00").time()
preopen_end = pd.to_datetime("09:15:00").time()
df_preopen = df[(df['time'] >= preopen_start) & (df['time'] < preopen_end)].copy()

# === Compute Sentiment Metrics per Symbol ===
summary = df_preopen.groupby('symbol').agg(
    total_bQty=('bQty', 'sum'),
    total_sQty=('sQty', 'sum'),
    avg_price=('last', 'mean'),
    first_price=('last', 'first'),
    last_price=('last', 'last'),
    total_volume=('ltq', 'sum')
)

summary['gap_percent'] = ((summary['last_price'] - summary['first_price']) / summary['first_price']) * 100
summary['demand_ratio'] = summary['total_bQty'] / (summary['total_sQty'] + 1)

def classify_sentiment(row):
    if row['gap_percent'] > 0.5 and row['demand_ratio'] > 1.5:
        return "📈 Bullish"
    elif row['gap_percent'] < -0.5 and row['demand_ratio'] < 0.7:
        return "📉 Bearish"
    return "⚖️ Neutral"

summary['sentiment'] = summary.apply(classify_sentiment, axis=1)
summary = summary.sort_values(by='total_volume', ascending=False).head(20)

# === Display ===
print("\n🕒 PRE-OPEN SENTIMENT FROM TICK DATA (9:00–9:15)")
print(summary[['gap_percent', 'demand_ratio', 'total_volume', 'sentiment']])


✅ Extracted to: extracted
📄 Files:
 - 18-07-2025_0.txt

🕒 PRE-OPEN SENTIMENT FROM TICK DATA (9:00–9:15)
            gap_percent  demand_ratio  total_volume   sentiment
symbol                                                         
1.1!533090    -2.479339  1.781933e+01       6556396  ⚖️ Neutral
1.1!531723     0.000000  4.798863e+00       3500311  ⚖️ Neutral
1.1!531205    -8.333333  1.532280e+00       3103776  ⚖️ Neutral
1.1!517554     0.000000  2.771252e+01       2855399  ⚖️ Neutral
1.1!524661    -0.360360  3.855660e-05       1546857  ⚖️ Neutral
1.1!532368     0.000000  1.062476e-07       1186555  ⚖️ Neutral
1.1!532822    -0.891720  1.545588e+00        515149  ⚖️ Neutral
1.1!523277     0.000000  8.913450e-01        363384  ⚖️ Neutral
1.1!532627     0.132159  2.751975e+00        306563  ⚖️ Neutral
1.1!532215    -5.640485  9.002506e-02        252724   📉 Bearish
1.1!530095     0.000000  1.074194e-02        175362  ⚖️ Neutral
1.1!503641    -6.185567  5.365723e-02        162677   📉 Bearish


In [ ]:
import pandas as pd
import numpy as np
import glob
import os

import zipfile
import os

# === Path to your ZIP file ===
zip_path = "18-07-2025_0.zip"  # Ensure it's in the same folder or provide full path
extract_to = "extracted"

# === Create folder if needed ===
os.makedirs(extract_to, exist_ok=True)

# === Extract all files ===
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# === List extracted files ===
print(f"✅ Extracted to: {extract_to}")
print("📄 Files:")
for file in os.listdir(extract_to):
    print(" -", file)


# === 🔄 Load and Combine All Market Files ===
folder = "extracted"  # Or specify full path
files = glob.glob(os.path.join(folder, "18-07-2025_*.txt"))
columns = [
    "symbol", "open", "last", "high", "low", "change", "bPrice", "bQty", "sPrice", "sQty",
    "ltq", "avgPrice", "quotes", "ttq", "totalBuyQt", "totalSellQ", "ttv", "trend",
    "lowerCktLm", "upperCktLm", "ltt", "close", "exchange", "stock_name", "VolumeC", "OI",
    "CHNGOI", "product_type", "expiry_date", "strike_price"
]

df_list = []
for file in files:
    df_temp = pd.read_csv(file, header=None, names=columns)
    df_list.append(df_temp)

df = pd.concat(df_list, ignore_index=True)

# === Convert numeric columns ===
# Include all columns used in calculations that should be numeric
numeric_cols = ['open', 'last', 'high', 'low', 'change', 'bPrice', 'bQty', 'sPrice',
                'sQty', 'ltq', 'avgPrice', 'quotes', 'ttq', 'totalBuyQt', 'totalSellQ',
                'ttv', 'VolumeC', 'OI', 'CHNGOI', 'lowerCktLm', 'upperCktLm', 'close']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Drop duplicates and sort after numeric conversion
df = df.drop_duplicates().sort_values(by=['symbol', 'ltt'])
df['ltt'] = pd.to_datetime(df['ltt'], errors='coerce')
df = df[df['ltt'].notnull()]


# === Feature Engineering ===
def analyze_symbol(df_sym):
    df_sym = df_sym.set_index('ltt').copy()
    # Ensure columns used in calculations are numeric
    numeric_cols_sym = ['last', 'bQty', 'sQty', 'ltq', 'ttv', 'high', 'low', 'VolumeC']
    df_sym[numeric_cols_sym] = df_sym[numeric_cols_sym].apply(pd.to_numeric, errors='coerce')


    df_sym['delta_price'] = df_sym['last'].diff()
    df_sym['rolling_bQty'] = df_sym['bQty'].rolling(30).mean()
    df_sym['rolling_sQty'] = df_sym['sQty'].rolling(30).mean()
    df_sym['rolling_ltq'] = df_sym['ltq'].rolling(30).mean()
    df_sym['rolling_price'] = df_sym['last'].rolling(30).mean()
    # Handle potential division by zero in order_imbalance
    df_sym['order_imbalance'] = df_sym['rolling_bQty'] / (df_sym['rolling_sQty'] + df_sym['rolling_bQty']).replace(0, np.nan)

    # Handle potential division by zero in vwap and ensure sum is numeric
    ttv_cumsum = df_sym['ttv'].cumsum()
    ltq_cumsum = df_sym['ltq'].cumsum()
    df_sym['vwap'] = ttv_cumsum / (ltq_cumsum + 1).replace(0, np.nan)


    df_sym['volume_spike'] = df_sym['ltq'] > df_sym['rolling_ltq'] * 2
    df_sym['price_above_vwap'] = df_sym['last'] > df_sym['vwap']

    # Score system
    score = 0
    if df_sym['order_imbalance'].mean() > 1.5:
        score += 1
    if df_sym['price_above_vwap'].mean() > 0.6:
        score += 1
    if df_sym['volume_spike'].sum() > 5:
        score += 1
    if df_sym['delta_price'].mean() > 0:
        score += 1

    if score >= 3:
        sentiment = "📈 Strong Bullish"
    elif score == 2:
        sentiment = "⚖️ Mixed to Bullish"
    elif score == 1:
        sentiment = "⚠️ Weak"
    else:
        sentiment = "📉 Bearish"

    return {
        "symbol": df_sym['symbol'].iloc[0],
        "score": score,
        "sentiment": sentiment,
        "imbalance": df_sym['order_imbalance'].mean(),
        "avg_delta_price": df_sym['delta_price'].mean()
    }

result = []
for symbol in df['symbol'].unique():
    df_sym = df[df['symbol'] == symbol].copy()
    if len(df_sym) < 60: continue
    result.append(analyze_symbol(df_sym))

summary = pd.DataFrame(result)
summary = summary.sort_values(by="score", ascending=False)

print("\n🧠 TICK-BASED MARKET STRATEGY ANALYSIS")
print(summary[['symbol', 'score', 'sentiment', 'imbalance', 'avg_delta_price']].head(20))

# Composite Sentiment
strong_bullish = (summary['sentiment'] == '📈 Strong Bullish').sum()
bearish = (summary['sentiment'] == '📉 Bearish').sum()

print(f"\n✅ Strong Bullish Symbols: {strong_bullish}")
print(f"❌ Bearish Symbols: {bearish}")

if strong_bullish > bearish * 1.5:
    print("📊 Final Market View: STRONG BULLISH BIAS for tomorrow")
elif bearish > strong_bullish * 1.5:
    print("📊 Final Market View: STRONG BEARISH BIAS for tomorrow")
else:
    print("📊 Final Market View: MIXED or RANGE-BOUND sentiment")

✅ Extracted to: extracted
📄 Files:
 - 18-07-2025_0.txt

🧠 TICK-BASED MARKET STRATEGY ANALYSIS
         symbol  score            sentiment  imbalance  avg_delta_price
196  1.1!500464      3     📈 Strong Bullish   0.612190         0.166667
585  1.1!514440      3     📈 Strong Bullish   0.971475         0.359679
720  1.1!523367      2  ⚖️ Mixed to Bullish   0.464392         0.000417
721  1.1!523369      2  ⚖️ Mixed to Bullish   0.277202         0.348880
691  1.1!522195      2  ⚖️ Mixed to Bullish   0.557687         0.359284
693  1.1!522217      2  ⚖️ Mixed to Bullish   0.420586         0.031695
694  1.1!522229      2  ⚖️ Mixed to Bullish   0.828505         0.047704
719  1.1!523323      2  ⚖️ Mixed to Bullish   0.597045         0.046154
378  1.1!506414      2  ⚖️ Mixed to Bullish   0.899961         0.013043
380  1.1!506525      2  ⚖️ Mixed to Bullish   0.117880         0.089259
381  1.1!506528      2  ⚖️ Mixed to Bullish   0.324290        33.395349
382  1.1!506579      2  ⚖️ Mixed to Bullis

In [ ]:
import pandas as pd
import numpy as np
import glob
import os

import zipfile
import os

# === Path to your ZIP file ===
zip_path = "18-07-2025_0.zip"  # Ensure it's in the same folder or provide full path
extract_to = "extracted"

# === Create folder if needed ===
os.makedirs(extract_to, exist_ok=True)

# === Extract all files ===
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# === List extracted files ===
print(f"✅ Extracted to: {extract_to}")
print("📄 Files:")
for file in os.listdir(extract_to):
    print(" -", file)


# === 🔄 Load and Combine All Market Files ===
folder = "extracted"  # Or specify full path
files = glob.glob(os.path.join(folder, "18-07-2025_*.txt"))

columns = [
    "symbol", "open", "last", "high", "low", "change", "bPrice", "bQty", "sPrice", "sQty",
    "ltq", "avgPrice", "quotes", "ttq", "totalBuyQt", "totalSellQ", "ttv", "trend",
    "lowerCktLm", "upperCktLm", "ltt", "close", "exchange", "stock_name", "VolumeC", "OI",
    "CHNGOI", "product_type", "expiry_date", "strike_price"
]

df_list = []
for file in files:
    df_temp = pd.read_csv(file, header=None, names=columns)
    df_list.append(df_temp)

df = pd.concat(df_list, ignore_index=True).drop_duplicates()
df['ltt'] = pd.to_datetime(df['ltt'], errors='coerce')
df = df[df['ltt'].notnull()].sort_values(by=['symbol', 'ltt'])

# === Strategy Engine ===
def analyze_symbol(df_sym):
    df_sym = df_sym.set_index('ltt').copy()

    # Explicitly convert columns to numeric within the function
    numeric_cols_sym = ['open', 'last', 'high', 'low', 'bPrice', 'bQty', 'sPrice', 'sQty',
                        'ltq', 'ttv', 'VolumeC', 'change'] # Added 'change'
    df_sym[numeric_cols_sym] = df_sym[numeric_cols_sym].apply(pd.to_numeric, errors='coerce')


    df_sym['delta_price'] = df_sym['last'].diff()
    df_sym['cumulative_delta'] = (df_sym['bQty'] - df_sym['sQty']).cumsum()
    df_sym['rolling_bQty'] = df_sym['bQty'].rolling(30).mean()
    df_sym['rolling_sQty'] = df_sym['sQty'].rolling(30).mean()
    df_sym['rolling_ltq'] = df_sym['ltq'].rolling(30).mean()
    df_sym['order_imbalance'] = df_sym['rolling_bQty'] / (df_sym['rolling_sQty'] + 1)
    df_sym['vwap'] = df_sym['ttv'].cumsum() / (df_sym['ltq'].cumsum() + 1)
    df_sym['price_above_vwap'] = df_sym['last'] > df_sym['vwap']
    df_sym['delta_divergence'] = df_sym['delta_price'] * df_sym['cumulative_delta']
    df_sym['speed_tape'] = df_sym['ltq'].diff().abs() > df_sym['rolling_ltq'] * 1.5

    # Smart Score
    score = 0
    if df_sym['order_imbalance'].mean() > 1.5:
        score += 1  # Order Imbalance
    if df_sym['price_above_vwap'].mean() > 0.6:
        score += 1  # VWAP Reclaim
    if df_sym['delta_divergence'].mean() > 0:
        score += 1  # Delta Divergence
    if df_sym['speed_tape'].sum() > 5:
        score += 1  # Speed of Tape

    if score == 4:
        sentiment = "🚀 Institutional Buy Setup"
    elif score == 3:
        sentiment = "📈 High Confidence Bullish"
    elif score == 2:
        sentiment = "⚖️ Cautious / Mixed"
    elif score == 1:
        sentiment = "⚠️ Weak Signal"
    else:
        sentiment = "📉 Likely Institutional Selling"

    return {
        "symbol": df_sym['symbol'].iloc[0],
        "score": score,
        "sentiment": sentiment,
        "imbalance": df_sym['order_imbalance'].mean(),
        "divergence": df_sym['delta_divergence'].mean(),
        "speed_tape_hits": df_sym['speed_tape'].sum()
    }

# === Run Analysis ===
results = []
for symbol in df['symbol'].unique():
    df_sym = df[df['symbol'] == symbol].copy()
    if len(df_sym) >= 60:
        results.append(analyze_symbol(df_sym))

summary = pd.DataFrame(results).sort_values(by='score', ascending=False)

# === Output ===
print("\n🏦 SMART INSTITUTIONAL MODEL v2 RESULTS:")
print(summary[['symbol', 'score', 'sentiment', 'imbalance', 'divergence', 'speed_tape_hits']].head(25))

print("\n📊 Sentiment Summary:")
print(summary['sentiment'].value_counts())

✅ Extracted to: extracted
📄 Files:
 - 18-07-2025_0.txt

🏦 SMART INSTITUTIONAL MODEL v2 RESULTS:
          symbol  score                  sentiment  imbalance   divergence  \
4     1.1!500012      3  📈 High Confidence Bullish   5.223558   163.215084   
1151  1.1!543929      3  📈 High Confidence Bullish   1.527578     2.572685   
738   1.1!523610      3  📈 High Confidence Bullish   1.610901    24.476221   
740   1.1!523630      3  📈 High Confidence Bullish   1.759281    20.318619   
742   1.1!523648      3  📈 High Confidence Bullish   4.584750    43.638150   
746   1.1!523696      3  📈 High Confidence Bullish   1.958507    38.397597   
752   1.1!523792      3  📈 High Confidence Bullish   1.533290    18.637609   
753   1.1!523828      3  📈 High Confidence Bullish  22.702062    33.948913   
38    1.1!500096      3  📈 High Confidence Bullish   2.234532    79.786827   
44    1.1!500108      3  📈 High Confidence Bullish   1.567137     8.169625   
45    1.1!500109      3  📈 High Confidence Bul

In [ ]:
import pandas as pd
import numpy as np
import glob
import os

import zipfile
import os

# === Path to your ZIP file ===
zip_path = "18-07-2025_0.zip"  # Ensure it's in the same folder or provide full path
extract_to = "extracted"

# === Create folder if needed ===
os.makedirs(extract_to, exist_ok=True)

# === Extract all files ===
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# === List extracted files ===
print(f"✅ Extracted to: {extract_to}")
print("📄 Files:")
for file in os.listdir(extract_to):
    print(" -", file)


# === 🔄 Load and Combine All Market Files ===
folder = "extracted"  # Or specify full path
files = glob.glob(os.path.join(folder, "18-07-2025_*.txt"))

columns = [
    "symbol", "open", "last", "high", "low", "change", "bPrice", "bQty", "sPrice", "sQty",
    "ltq", "avgPrice", "quotes", "ttq", "totalBuyQt", "totalSellQ", "ttv", "trend",
    "lowerCktLm", "upperCktLm", "ltt", "close", "exchange", "stock_name", "VolumeC", "OI",
    "CHNGOI", "product_type", "expiry_date", "strike_price"
]

df_list = []
for file in files:
    df_temp = pd.read_csv(file, header=None, names=columns)
    df_list.append(df_temp)

df = pd.concat(df_list, ignore_index=True).drop_duplicates()
df['ltt'] = pd.to_datetime(df['ltt'], errors='coerce')
df = df[df['ltt'].notnull()].sort_values(by=['symbol', 'ltt'])

# === Define Strategy Function ===
def evaluate_strategies(df_sym):
    df_sym = df_sym.set_index('ltt').copy()

    # Explicitly convert columns to numeric within the function
    numeric_cols_sym = ['open', 'last', 'high', 'low', 'bPrice', 'bQty', 'sPrice', 'sQty',
                        'ltq', 'ttv', 'VolumeC']
    df_sym[numeric_cols_sym] = df_sym[numeric_cols_sym].apply(pd.to_numeric, errors='coerce')

    df_sym['delta_price'] = df_sym['last'].diff()
    df_sym['rolling_bQty'] = df_sym['bQty'].rolling(30).mean()
    df_sym['rolling_sQty'] = df_sym['sQty'].rolling(30).mean()
    df_sym['rolling_ltq'] = df_sym['ltq'].rolling(30).mean()
    df_sym['order_imbalance'] = df_sym['rolling_bQty'] / (df_sym['rolling_sQty'] + 1)
    df_sym['vwap'] = df_sym['ttv'].cumsum() / (df_sym['ltq'].cumsum() + 1)
    df_sym['volume_spike'] = df_sym['ltq'] > df_sym['rolling_ltq'] * 2
    df_sym['price_above_vwap'] = df_sym['last'] > df_sym['vwap']

    # Strategy Scores (1 point per satisfied condition)
    score = 0
    strategies = {}

    # 1. Order Imbalance
    strategies['order_imbalance'] = df_sym['order_imbalance'].mean()
    if strategies['order_imbalance'] > 1.5:
        score += 1

    # 2. Delta Tick Logic
    strategies['avg_delta_price'] = df_sym['delta_price'].mean()
    if strategies['avg_delta_price'] > 0:
        score += 1

    # 3. VWAP Bounce
    above_pct = df_sym['price_above_vwap'].mean()
    strategies['price_above_vwap_pct'] = above_pct
    if above_pct > 0.6:
        score += 1

    # 4. Volume Spike
    spike_count = df_sym['volume_spike'].sum()
    strategies['volume_spike_count'] = spike_count
    if spike_count > 5:
        score += 1

    # 5. Time-of-day trap detection (simplified for afternoon flush)
    df_sym['hour'] = df_sym.index.hour
    # Ensure 'ltq' is numeric before summing
    trap_volume = df_sym.between_time("14:30", "14:45")["ltq"].apply(pd.to_numeric, errors='coerce').sum()
    if trap_volume < df_sym['ltq'].mean() * 15:  # Low vol in flush window
        score += 1

    # 6. Trend Confirmation
    # Ensure 'delta_price' is numeric before calculating mean
    if (df_sym['delta_price'].tail(60).apply(pd.to_numeric, errors='coerce') > 0).mean() > 0.7:
        score += 1

    # 7. Exploding Tape
    # Ensure 'delta_price' and 'last' are numeric
    explosive_moves = df_sym['delta_price'].apply(pd.to_numeric, errors='coerce').abs() > df_sym['last'].apply(pd.to_numeric, errors='coerce').mean() * 0.003
    if explosive_moves.sum() > 10:
        score += 1

    # 8. Smart Signal Combo
    if score >= 5:
        sentiment = "📈 Strong Bullish"
    elif score >= 3:
        sentiment = "⚖️ Mixed to Bullish"
    elif score == 2:
        sentiment = "⚠️ Weak"
    else:
        sentiment = "📉 Bearish"

    strategies['score'] = score
    strategies['sentiment'] = sentiment
    strategies['symbol'] = df_sym['symbol'].iloc[0]
    return strategies

# === Run on All Symbols ===
results = []
for symbol in df['symbol'].unique():
    df_sym = df[df['symbol'] == symbol].copy()
    if len(df_sym) >= 60:
        result = evaluate_strategies(df_sym)
        results.append(result)

summary = pd.DataFrame(results).sort_values(by="score", ascending=False)

# === Display Top ===
print("\n🎯 COMPOSITE STRATEGY RESULT:")
print(summary[['symbol', 'score', 'sentiment', 'order_imbalance', 'avg_delta_price']].head(20))

# === Final Sentiment Summary ===
print("\n📊 Sentiment Count:")
print(summary['sentiment'].value_counts())

✅ Extracted to: extracted
📄 Files:
 - 18-07-2025_0.txt

🎯 COMPOSITE STRATEGY RESULT:
          symbol  score         sentiment  order_imbalance  avg_delta_price
803   1.1!524717      5  📈 Strong Bullish         1.912149         4.903226
436   1.1!508980      5  📈 Strong Bullish         4.264507         0.110867
822   1.1!526193      5  📈 Strong Bullish         2.132829         0.213529
438   1.1!509048      5  📈 Strong Bullish         6.368147         0.149804
570   1.1!514175      5  📈 Strong Bullish         1.802400         0.000853
440   1.1!509055      5  📈 Strong Bullish         1.642920         0.000814
969   1.1!531746      5  📈 Strong Bullish        11.305515         0.070365
447   1.1!509486      5  📈 Strong Bullish         7.148245         1.124427
418   1.1!507828      5  📈 Strong Bullish         2.876359         0.002974
377   1.1!506405      5  📈 Strong Bullish         3.912447         0.000881
1048  1.1!532740      5  📈 Strong Bullish         1.616080         0.240180
378